   - 학습 데이터 (user_id : 10000 ~ 24999, 15000명)


			- train_err_data.csv : 시스템에 발생한 에러 로그

			- train_quality_data.csv : 시스템 퀄리티 로그

			- train_problem_data.csv : 사용자 불만 및 불만이 접수된 시간


   - 테스트 데이터(user_id : 30000 ~ 44998, 14999명)


			- test_err_data.csv : 시스템에 발생한 에러 로그

			- test_quality_data.csv : 시스템 퀄리티 로그

			- sample_submission.csv : 사용자 불만 확률(0~1) (제출용)

## 사용할 Module & Data 불러오기 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
import seaborn as sns
from sklearn.metrics import *
from sklearn.model_selection import KFold
import missingno as msno
import warnings
warnings.filterwarnings(action='ignore')

In [15]:
PATH = '../data/'
train_err =pd.read_csv('../data/train_err.csv')
train_problem =pd.read_csv('../data/train_problem.csv')
train_quality =pd.read_csv('../data/train_quality.csv')
test_err = pd.read_csv('../data/test_err.csv')
test_quality = pd.read_csv('../data/test_quality.csv')


# err data 는 user_id 15000 명의 대한것이다

### time datetime으로 바꾸기

In [3]:
# time 을 datetime 타입으로 바꾸고 바뀐지 확인
# train_err.info()
# test_err.info()
# test_quality.info()
# train_quality.info()
# train_problem.info()
# train_problem.time = train_problem.time.map(make_datetime)
# train_quality.time = train_quality.time.map(make_datetime)
# test_quality.time = test_quality.time.map(make_datetime)

In [14]:
# 바꾼 csv 저장

# train_quality.to_csv('../data/train_quality.csv')
# train_problem.to_csv('../data/train_problem.csv')
# test_quality.to_csv('../data/test_quality.csv')


In [16]:
# time 을 datetime 타입으로 바꾸고 바뀐지 확인
train_quality.time = pd.to_datetime(train_quality.time)
test_err.time = pd.to_datetime(test_err.time)
train_problem.time = pd.to_datetime( train_problem.time)
train_err.time = pd.to_datetime(train_err.time)
test_quality.time = pd.to_datetime(test_quality.time)


In [17]:
# pd.to_datetime(train_err_df.time)
# pd.set_option('display.max_rows)
# pandas display 할때 보여주는 최대 로우수와 컬럼수 지정  
pd.options.display.max_rows = 70
pd.options.display.max_columns = 30

In [ ]:
def make_datetime(x):
    x = str(x)
    year = int(x[:4])
    month = int(x[4:6])
    day = int(x[6:8])
    hour = int(x[8:10])
    mim = int(x[10:12])
    sec = int(x[12:])
    return dt.datetime(year,month,day,hour,mim,sec)
# make_datetime(20201101025616)
# train_err_df['time']= train_err_df.time.map(make_datetime)

In [ ]:
dt.MINYEAR
dt.MAXYEAR


## EDA

- 불만접수를 err 데이터의 있는 모든 유저 가 불만접수를 했는지 확인

In [ ]:
train_err_df.user_id.value_counts()
train_problem_df.user_id.value_counts()
train_quality_df.user_id.value_counts()
#quality user_id 8281
#problem user_id 5000
#err user_id     15000
#에러가 발생한 모든 유저가 불만 접수를 한것이 아님
# 그러면 특정 유저

### 0609 한것

In [ ]:
train_quality_df.info() 
train_problem_df.info()
train_err_df.info()

In [ ]:
null_columns = train_quality_df.columns[train_quality_df.isnull().sum() > 0].to_list()
# 이거 왜 되지 ;; 여러개 의 행의 빈값 있는 것만 찾으려했는데 조건식 만들기가 힘드네 
for column in null_columns:
    display(train_quality_df[train_quality_df[column].isnull()])

In [ ]:
train_err_df.set_index(['user_id','errtype','model_nm']).loc[10000,'errcode']

In [ ]:
np.sort(train_err_df.errtype.unique())
# 29제외 1부터 42까지 41가지 존재한다.

In [ ]:
id_error = train_err_df[['user_id','errtype']].values
id_error

In [ ]:
sns.heatmap(train_err_df.isnull(), cbar=False)

In [ ]:
msno.matrix(train_err_df)

In [ ]:
train_quality_df[train_quality_df.user_id == 10693]

In [ ]:
train_err_df[train_err_df.user_id == 10693]
# user_id 10693 은 model_6 만 사용했는지는 모르지만 error 이것만떳어 fwver 이 10 이라고 error 데이터에서는 보이는 그러면
# error 가 발생하고 quality 를 2시간동안 10분마다 수집하니까  근데 보면 20일에 발생한 에러는 없어 에러를 거의 2 분 3분마다 발생시키는데 
# 무슨 에러인지 모르겠네 어차피 이건 model_6 fwver 10에 대한 에러들이야

In [ ]:
%%time
# quailty 에서 fwver 가 nan인게 뭐뭐가 있는지 fwver 가 빈것들은 다 퀄리티 0 과 2가 비어있는지 봐보자
train_quality_df.loc[train_quality_df.fwver.isnull(),['quality_0','quality_2']].notnull().sum()
# 다 비어 있다 그러면 모델이 같은지 한번 봐보자
null_qual_0_1 = train_quality_df.loc[train_quality_df.fwver.isnull(),'user_id'].unique() # 개수는 59개

def isin_null_0_1(x):
    if x in null_qual_0_1:
        return True
    else:
        return False
null_0_1 = train_err_df[train_err_df.user_id.map(isin_null_0_1)]
'''
model_6    1639
model_0    1630
model_2     783
model_3      43
'''
null_0_1[null_0_1.model_nm == "model_6"].fwver.value_counts()
#array(['model_6', 'model_3', 'model_0', 'model_2'], dtype=object)

In [ ]:
# model 종류가 몇가지나 있나 한번 봐보자 
train_err_df.model_nm.value_counts()
# 빈것들의 에러코드는 같은지 or model 이 같은지 
# 퀄리티 별 숫자가 같은지 로봐보자 

In [ ]:
train_quality_df[train_quality_df.columns[3:]].iloc[:,0].value_counts(normalize=True)

In [ ]:
# null_0_1.errtype.value_counts()
# model_nm 과 error 타입을 연결시키는건 좀 힘들고
# user_id 별로 무슨 error type 이나 error code에 대한 불만을 많이 썻는지
# 그 에러들의 퀄리티 점수 분포는 어떻게 되는지 로 보면 될거같은데
# 결측 치 채우기는 다음에 위에꺼 부터
plt.figure(figsize =(10,5))
train_problem_df.user_id.value_counts().hist()
# plt.ylim(0,100)
plt.show()
# 대부분 의 고객이 불만은 한번 표시 했다

### 0610

In [ ]:
train_err.info()
train_problem_df.info()
train_quality_df.info()

In [ ]:
#중복값 확인해서 똑같은걸 처리해준다
#왜냐 왜냐 음 왜일까 수학적으로 가중치를 더주게 될거라는 느낌은 무엇을 보고 느낀걸까 아마 그럴거라고 듣기만해서 일텐데 
#오차를 구하는 방식에서 똑같은 것만 있다면 우린 loss를 줄이는 방식인가 ? lgbm 이 어떤식으로 작동했던건지 기억이 안나서 잘안돼네

# train_err_df # 16553663row
# train_err= train_err_df.drop_duplicates(keep= 'last', ignore_index = True)
# train_problem_df.duplicated().sum()# 중복값 불만접수 데이터에는 없고

# 어 그러면 2시간 동안 한번씩 찍힌다고 했는데 한번만 찍힌것도 있나
# train_quality_df[train_quality_df.time.duplicated(keep=False)]
# 확인 결과 없고 당연하네

# dup_user = train_quality_df.loc[:,'user_id'].unique()
# dup_user # 처음 중복되는 값을 여기도 확인하려했었는데 반복된다고했고 time 을 에러가 뜬순간으로 처리해놔서 구별이 되게했구나 아아아아아 분 초를 없앴어
# 분초를 진짜 없앴는지 봐보자
#train_err # 일단 에러는 시분초 다 세세히 적혀있어

# train_quality_df.time.map(lambda x: x.minute).value_counts()
# quality는
'''
20    151224
0     145920
10    140652
30    133896
50    131316
40    125616
'''
# train_quality_df.time.map(lambda x: x.second).value_counts()
#0    828624 초는 다없애고 분은 10분 단위에 맞춰서 한거같다 하나하나 비교해야되는데 능력부족

# train_quality_df[train_quality_df['user_id'] ==dup_user[2]]
# 이건 에러가 뜨고 quality_0~12 가 점수가 같은지 같은에러에는 같은지 봐볼라고했다 
# 에러가 같은건 같은 시간대에 같은 user_id fwver 으로 같은 에러라고 판단했고
# 거의 같은 양상을 보이지만 완전히 같지 않기에 항상 quality가 똑같다고 볼수없다 에러가 안떳는데도 퀄리티가 찍히는 데이터는 아닌거같은데 확인은 해봐야겠고






# 품질 이 양수와 음수를 보고 품질이 음수일때 불만 접수를 하는지 
# 아 근데 에러가 떳을때 quality log 를 찍는단 말이야 그러면 그래도 의미 있을거같애
# 모델이 뭔지를 안다면 안찍히는것은 기능 유추를 해볼수 있을텐데 그것도 모르겠고
# LG 는 가전제품 생산하는 회사야 그러면 log 를 찍을수있는건가 ? 당연한건가 품질로그를?

# 불만을 표한 error 가 무엇일까 그걸 찾고 그 순간에 퀄리티 점수 는 ? ㅇㅋ 이걸로 간다 
# merge_df = pd.merge(train_problem_df,train_err , on = 'user_id')
# merge_df[merge_df['user_id'] == 19224][:100]
# 19224 	2020-11-02 20:00:00

# outlier 여부 확인도 해야겠네
# quality 유저별 펌웨어별 그리고 시간별 평균 합  분산 ? ㅇ
train_err.to_csv('train_err.csv' , index = False)

In [ ]:
train_errtype = np.sort(train_err.errtype.unique())
train_errtype
test_errtype = np.sort(test_err.errtype.unique())
test_errtype == train_errtype

In [ ]:
train_user_id_max= 14999
train_user_id_min= 10000
train_user_id_num= 15000

labels = np.zeros(train_user_id_num)
# print(labels.dtype)
labels[train_problem_df.user_id.unique()-10000] = 1
labels = labels.astype('int16')

def labeling(x):
    return labels[x -10000]

train_err['problem'] = train_err.user_id.map(labeling)
train_err
train_quality_df['problem']= train_quality_df.user_id.map(labeling)
train_quality_df
# 각 user_id 보고 거기다 라벨 붙이기 

In [ ]:
# re_number = re.compile("\d")
# re_number.search('6a')
# 확실히 에러 가 뜬다고 다 적은게 아니다 결측값이 있다 
# 없어 불리 할수밖에없다 
# 합쳐져 사용하고싶다 빈게 많을것이다 맞냐 틀리냐 
# 빈것을 어떻게 합쳐서 쓰냐 하면
# train Err 랑 problem 만쓴거 보다 성능이 안좋다 그러면 
# 여러개 시간별로 다양한 데이터가 있다 그러면 어떻게 사용할것이냐 ?
# 이데이터에 대해서 얼마나 이해하고있는지 고민해보고 
# 왜 이문제를 선택했는지 
# 시간 차이 테스트 랑 트레인 
# 시간차이를 봐봐야돼 퀄리티랑 에러도 보고 그렇게 다봐봐야돼
# 원하는 데이터 타입의 컬럼만 뽑기
# train_quality_df.select_dtypes('object')

fig, axs = plt.subplots(1,3, figsize = (21,7))

sns.heatmap(train_err.corr(), ax= axs[0], annot = True)
sns.heatmap(train_problem_df.corr(), ax= axs[1], annot = True)
sns.heatmap(train_quality_df.corr(), ax= axs[2], annot = True)

In [ ]:
train_quality_df.corr().iloc[:,2:]
# 영향을 끼친다를 어떻게 판단하는지 영향이 있다 느껴지면 그걸 어떻게 활용하는지 모르겠다
train_quality_df.describe()
train_quality_df.info()

In [ ]:
# train_quality_df['quality_5'].apply(lambda x :  0 if type(x) != str and math.isnan(x)   else int(x.replace(',','')) )
# train_quality_df[train_quality_df.iloc[:,8].isna()]
# type(train_quality_df.loc[58907].loc['quality_5'])
train_quality_df.select_dtypes('object').iloc[:,2].apply(lambda x :  0 if (type(x) != str and math.isnan(x)) int(x.replace(',','')) elif type(x) == str else x)

In [ ]:
import math
import numpy as np

In [ ]:
# %%time

# b = math.nan
# print(np.isnan(b))

# a = np.nan
# print(np.isnan(a))

# (lambda x : 0.0 if math.isnan(x) else x)(train_quality_df.loc[58907].loc['quality_5'])
# train_quality_df.loc[58907].loc['quality_5']
type('str') == str

In [ ]:
%%time

b = math.nan
print(math.isnan(b))

a = np.nan
print(math.isnan(0.1))



## Data Preprocessing

### 초기 Errtype만을 이용하여 데이터를 학습시키기는 것으로 시작
- label 은 problem set 이 불만 접수를 한 유저 이다
  시계열 데이터인 이 데이터에서 시간 에 대한 활용은 아직 못하였기때문에 시간은 사용하지않음
  그러므로 problem 의 존재하는 user_id 불만접수 유저 label = 1로 지정 나머지는 0 

- user_id 별로 errtype 별 발생 횟수만을 학습 데이터로 전처리하여 학습시킨다

In [18]:
# train_err_data 의 유저 아이디는 15000명이고 10000~24999이기 때문에
# 더 연산이 빠르게 ndarray를 새로만들어서 0을 user_id 10000으로 해서 레이블을 붙인다
def make_label(df):
    problem = np.zeros(15000)
    problem[df.user_id -10000] = 1
    return problem
# np.zeros(15000)
# train_problem[train_problem.user_id==10001]

problem = make_label(train_problem)
train_y = problem

In [19]:
# len(np.sort(train_err.errtype.unique())) # 이전 EDA에서도 봤지만 1~42 중 29가 빠진 41개의 errtype이 있다
user_errtype = np.zeros((15000,41))
user_errtype2= np.zeros((15000,42))
# user_errtype.shape

for user_id , type_num in tqdm(train_err[['user_id','errtype']].values):
    if type_num > 28:
        user_errtype[user_id-10000,type_num-2] += 1
    else:
        user_errtype[user_id-10000,type_num-1] += 1
    user_errtype2[user_id-10000,type_num-1] += 1
user_errtype , user_errtype2
# train_err[['user_id','errtype']].values
# user_errtype[10000-10000,30-2] += 1
# user_errtype[0,29]


100%|███████████████████████████| 15368002/15368002 [00:23<00:00, 661353.41it/s]


(array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ..., 113.,  56.,   1.],
        [  0.,   0.,   2., ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0., ...,  58.,   7.,   5.],
        [  0.,   0.,   0., ...,   6.,   0.,   0.],
        [  0.,   0.,   4., ...,   0.,   0.,   0.]]),
 array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ..., 113.,  56.,   1.],
        [  0.,   0.,   2., ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0., ...,  58.,   7.,   5.],
        [  0.,   0.,   0., ...,   6.,   0.,   0.],
        [  0.,   0.,   4., ...,   0.,   0.,   0.]]))

In [ ]:
# print(user_errtype2[0,2].sum())
# train_err[(train_err.errtype == 3) & (train_err.user_id == 10000)]
train_x = user_errtype2
print(train_x.shape)
print(train_y.shape)

In [22]:
# train_data = pd.DataFrame(user_errtype2)
# train_data2 = pd.DataFrame(user_errtype)
# train_label = pd.Series(train_y)

# train_data.to_csv('../data/train_data.csv', index= False)
# train_data2.to_csv('../data/train_data2.csv', index= False)
# train_label.to_csv('../data/label.csv', index= False)
PATH = '../data/'
train_x = pd.read_csv(PATH+'train_data.csv')
train_y = pd.read_csv(PATH+'label.csv')
train_y

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
14995,0.0
14996,0.0
14997,1.0
14998,1.0


## Modeling & Training

### LightBGM

In [ ]:
def f_pr_auc(probas_pred, y_true):
    labels = y_true.get_label()
    p ,r , _ = precision_recall_curve(labels, probas_pred)
    score = auc(r,p)
    return "pr_auc", score, True


In [ ]:
models = []
recalls = []
precisions = []
auc_scores = []
threshold = 0.5
# parameter 설정
params= {'boosting_type' : 'gbdt',
         'objective' : 'binary',
         'metric' : 'auc',
         'seed':1015
        }

k_fold = KFold(n_splits=5, shuffle=True, random_state = 42)
# 5 fold cross validation
for train_idx, val_idx in k_fold.split(train_x):
    
    #split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]
    
    d_train = lgb.Dataset(X,y)
    d_val = lgb.Dataset(valid_x, valid_y)
    
    # run training
    model = lgb.train(
        params,
        train_set = d_train , 
        num_boost_round = 1000, # num_trees 와 똑같이 보면된다
        valid_sets = d_val, 
        feval = f_pr_auc,  #customized evaluation function
        # verbose_eval = 20 ,
        early_stopping_rounds = 3
    )
    
    # cal valid prediction
    
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold,1,0)
    
    # cal scores
    
    recall = recall_score( valid_y, valid_pred)
    precision = precision_score(valid_y, valid_pred)
    auc_score = roc_auc_score(valid_y, valid_prob)
    
    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)
    
    print('='*38)

In [ ]:
# def funcSample():
#     print('sample')
# callable(funcSample)


### Xgboost

In [26]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_val , y_train ,y_val = train_test_split(
    train_x, train_y, test_size = 0.3 , random_state= 42)

# xgb.DMatrix(train_x,train_y)
X_train.shape , y_train.shape
X_val.shape , y_val.shape

((4500, 42), (4500, 1))

In [39]:
d_train = xgb.DMatrix(X_train,y_train)
d_val = xgb.DMatrix(X_val, y_val)
# xgb_model = xgb.XGBClassifier(silent=0)
# print(xgb.XGBClassifier())

In [41]:
params = {
    'booster' :'gbtree',
    'silent' : 0,
    'nthread': 10
}
xgb_model = xgb.train(params =params , dtrain = d_train, num_boost_round = 400,
         early_stopping_rounds= 100, evals = [(d_train,'train'),(d_val,'eval')])
# print("훈련 세트 정확도 : {:.3f}".format(xgb_model.score(X_train, y_train)))
# print("validation 세트 정확도 : {:.3f}".format(xgb_model.score(X_val, y_val)))
xgb_model


[17:10:35] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:0.45174	eval-rmse:0.45495
[1]	train-rmse:0.42356	eval-rmse:0.42984
[2]	train-rmse:0.40669	eval-rmse:0.41593
[3]	train-rmse:0.39585	eval-rmse:0.40770
[4]	train-rmse:0.38874	eval-rmse:0.40346
[5]	train-rmse:0.38262	eval-rmse:0.40056
[6]	train-rmse:0.37889	eval-rmse:0.39918
[7]	train-rmse:0.37541	eval-rmse:0.39893
[8]	train-rmse:0.37186	eval-rmse:0.39940
[9]	train-rmse:0.36894	eval-rmse:0.39910
[10]	train-rmse:0.36681	eval-rmse:0.39859
[11]	train-rmse:0.36365	eval-rmse:0.39904
[12]	train-rmse:0.36236	eval-rmse:0.39906
[13]	train-rmse:0.36011	eval-rmse:0.39889



## Test & measure performance

### 교차검증 점수 확인


In [ ]:
print(np.mean(auc_scores))

## 제출 파일 만들기


In [ ]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

In [ ]:
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
pred_ensemble

In [ ]:
sample_submssion = pd.read_csv('../data/sample_submission.csv')

sample_submssion['problem'] = pred_ensemble.reshape(-1)

sample_submssion.to_csv("../data/dacon_baseline.csv", index = False)
sample_submssion